In [1]:
import math
import torch
import gpytorch
import spectralgp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#import seaborn as sns
%matplotlib inline

#### Set up the GPyTorch Model with Spectral GP kernel
Using the same framework as standard GPyTorch models, we merely drop-in the spectral GP kernel as the covar module

The `initialize_from_data` method does some pre-training on the latent model using the log-periodogram of data as training targets.

For specifics on the components of GPyTorch models we refer to the [GPyTorch Documentation](https://gpytorch.readthedocs.io/en/latest/index.html)

In [2]:
class SpectralModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, **kwargs):
        super(SpectralModel, self).__init__(train_x, train_y, likelihood)

        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = spectralgp.kernels.SpectralGPKernel(**kwargs)
        self.covar_module.initialize_from_data(train_x, train_y, **kwargs)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


#### Generate training data and Build GP Model
Generate points in [0, 5] and a sine wave to serve as the response, then split into training and test data.

Pass this data into the GP model from above along with a likelihood

In [3]:
nx = 200
split = 150
full_x = torch.linspace(0, 5, nx)
full_y = torch.sin(2 * full_x)

train_x = full_x[:split]
train_y = full_y[:split]

test_x = full_x[(split - nx):]
test_y = full_y[(split - nx):]

In [4]:
likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_prior=gpytorch.priors.SmoothedBoxPrior(1e-8, 1e-4))
model = SpectralModel(train_x, train_y, likelihood, nomg=100)


None
Pretrain On


#### Set up sampling factories

In the inference procedure we consider fixing the latent GP observation and doing gradient descent updates on the hyperparameters, then fixing the hyperparameters and using elliptical slice sampling to update the latent GP. 

The `ss_factory` generates a "factory" that fixes the latent GP and computes the loss function of the hyperparameters

Set up Alternating Sampler

In [5]:
n_iters = 20
ess_iters = 10
optim_iters = 5

In [6]:
alt_sampler = spectralgp.samplers.AlternatingSampler(
    [model], [likelihood], 
    spectralgp.sampling_factories.ss_factory, [spectralgp.sampling_factories.ess_factory],
    totalSamples=n_iters, numInnerSamples=ess_iters, numOuterSamples=optim_iters
    )
alt_sampler.run();


Loss is:  tensor(-18.1137, grad_fn=<DivBackward0>)
Loss is:  tensor(-17.8779, grad_fn=<DivBackward0>)
Loss is:  tensor(-17.6377, grad_fn=<DivBackward0>)
Loss is:  tensor(-17.4017, grad_fn=<DivBackward0>)
Loss is:  tensor(-17.1742, grad_fn=<DivBackward0>)
Task: 0 ; Iteration 0
Seconds for Iteration 0 : 0.7656238079071045
Loss is:  tensor(-16.7839, grad_fn=<DivBackward0>)
Loss is:  tensor(-16.5562, grad_fn=<DivBackward0>)
Loss is:  tensor(-16.3355, grad_fn=<DivBackward0>)
Loss is:  tensor(-16.1146, grad_fn=<DivBackward0>)
Loss is:  tensor(-15.9021, grad_fn=<DivBackward0>)
Task: 0 ; Iteration 1
Seconds for Iteration 1 : 0.8067405223846436
Loss is:  tensor(-15.6566, grad_fn=<DivBackward0>)
Loss is:  tensor(-15.4383, grad_fn=<DivBackward0>)
Loss is:  tensor(-15.2311, grad_fn=<DivBackward0>)
Loss is:  tensor(-15.0225, grad_fn=<DivBackward0>)
Loss is:  tensor(-14.8192, grad_fn=<DivBackward0>)
Task: 0 ; Iteration 2
Seconds for Iteration 2 : 0.6222925186157227
Loss is:  tensor(-14.7524, grad_fn

#### Process the outputs
The alternating sampler provides a good set of hyperparamters (already attached to the model) and samples of the spectral density of the covariance function.

To generate predictions we look at the last samples of the alterntating sampler and generate covariance functions from each of these samples. 

In the below plotting we show 10 predictions: one prediction for each of the last 10 sampled log-spectral densities taken from the alternating sampler.

In [7]:
model.eval()
n_samples = 10
spectrum_samples = alt_sampler.gsampled[0][0,:, -10:].detach()

predictions = torch.zeros(len(full_x), 10) # predictions for each sample
upper_bds = torch.zeros(len(full_x), 10) # upper conf. bd for each sample
lower_bds = torch.zeros(len(full_x), 10) # lower conf. bd for each sample

with torch.no_grad():
    for ii in range(n_samples):
        model.covar_module.set_latent_params(spectrum_samples[:, ii])
        model.set_train_data(train_x, train_y) # to clear out the cache
        pred_dist = model(full_x) 
        lower_bds[:, ii], upper_bds[:, ii] = pred_dist.confidence_region()
        predictions[:, ii] = pred_dist.mean
        d = pred_dist.mean - full_y
        test_rmse = torch.sqrt(torch.mean(torch.pow(d, 2)))
        print("Normalised RMSE: {}".format(test_rmse))


Normalised RMSE: 0.1447228640317917
Normalised RMSE: 0.15227337181568146
Normalised RMSE: 0.14776091277599335
Normalised RMSE: 0.1967766135931015
Normalised RMSE: 0.16641126573085785
Normalised RMSE: 0.16823796927928925
Normalised RMSE: 0.17161661386489868
Normalised RMSE: 0.16461233794689178
Normalised RMSE: 0.16061799228191376
Normalised RMSE: 0.1534246951341629


In [13]:
print(len(alt_sampler.fgsampled))
print(alt_sampler.fgsampled[0].shape)

model.eval()

with torch.no_grad():
    for x in range(0,alt_sampler.fgsampled[0].shape[-1]):
        model.covar_module.set_latent_params(alt_sampler.fgsampled[0][0, :, x])
        model.set_train_data(train_x, train_y) # to clear out the cache
        pred_dist = model(full_x) 
        d = pred_dist.mean - full_y
        test_rmse = torch.sqrt(torch.mean(torch.pow(d, 2)))
        print("FULL DATASET: Normalised RMSE: {}".format(test_rmse))
        
#         pred_dist = model(test_x) 
#         d = pred_dist.mean - test_y
#         test_rmse = torch.sqrt(torch.mean(torch.pow(d, 2)))
#         print("TEST DATASET: Normalised RMSE: {}".format(test_rmse))
    


1
torch.Size([1, 100, 10])
FULL DATASET: Normalised RMSE: 0.1447228640317917
FULL DATASET: Normalised RMSE: 0.15227337181568146
FULL DATASET: Normalised RMSE: 0.14776091277599335
FULL DATASET: Normalised RMSE: 0.1967766135931015
FULL DATASET: Normalised RMSE: 0.16641126573085785
FULL DATASET: Normalised RMSE: 0.16823796927928925
FULL DATASET: Normalised RMSE: 0.17161661386489868
FULL DATASET: Normalised RMSE: 0.16461233794689178
FULL DATASET: Normalised RMSE: 0.16061799228191376
FULL DATASET: Normalised RMSE: 0.1534246951341629


#### Now Generate the Plot

In [ ]:
#sns.set_style("whitegrid")
colors = cm.get_cmap("tab10")
## plot the predictions ##
plt.plot(full_x.numpy(), predictions[:, 0].detach().numpy(), label="Predictions",
         color=colors(0), linewidth=2)
plt.plot(full_x.numpy(), predictions.detach().numpy(), linewidth=2, 
         color=colors(0))

## Shade region +/- 2 SD around the mean ##
plt.fill_between(full_x.numpy(), lower_bds[:, 0].detach().numpy(), 
                 upper_bds[:, 0].detach().numpy(),
                 color=colors(0), alpha=0.03, label = r"$\pm 2$ SD")
for ii in range(n_samples):
    plt.fill_between(full_x.numpy(), lower_bds[:, ii].detach().numpy(), 
                     upper_bds[:, ii].detach().numpy(), 
                     color=colors(0), alpha=0.03)
    
## plot data ##
plt.plot(train_x.numpy(), train_y.numpy(), color=colors(1),
        linewidth=2, label="Train Data")
plt.plot(test_x.numpy(), test_y.numpy(), color=colors(1),
        linestyle="None", marker=".", markersize=12,
        label="Test Data")
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Predictions and Data")
plt.legend()
plt.show()


In [ ]:
plt.plot(model.covar_module.omega.numpy(), spectrum_samples.exp().numpy(), label = 'Posterior Samples')
plt.xlabel('Omega')
plt.ylabel('Density')
plt.xlim((0, 7))
plt.ylim((0,1))
plt.vlines(2/(2*3.14159),ymin=0, ymax=10, label = 'True Period')
plt.legend()

In [ ]:
import numpy as np
print(np.trapz(spectrum_samples.exp().t().numpy(), model.covar_module.omega.numpy()))